# Build and Deploy

## Choosing Documentation Engine (MyST or Sphinx)

### MyST (Generally Recommended)
- Easy to add files
- Supports Jupyter notebooks (.ipynb)
- However, Jupyter Book v2 (which uses MyST) is still under development and has limited documentation

### Sphinx (Recommended for Experienced Users)
- Suitable if you have experience or need detailed customization
- Can build HTML locally
- Rich ecosystem of extensions and themes with extensive documentation
- Note: GitHub Actions builds don't support .ipynb files

The following sections use MyST as the base documentation engine.

## Customizing myst.yml Configuration

The generated `myst.yml` file can be customized to enhance your Jupyter Book documentation. Here are common customizations:

### Adding Files to Table of Contents

Edit the `toc:` section to include additional files:

```yaml
project:
  toc:
    - file: index.md
    - file: macro.md
    - title: Examples
      children:
        - file: example1.ipynb
        - file: example2.ipynb
    - title: Advanced Topics
      children:
        - file: advanced/topic1.md
        - file: advanced/topic2.md
```

### Adding a Logo

Add a logo image to your documentation:

```yaml
site:
  template: book-theme
  options:
    logo: logo.png
```

Place your logo file within your docs folder.

### Adding Action Buttons

You can configure links to SASPAC or PharmaForest by uncommenting the relevant sections:

```yaml
site:
  actions:
    - title: SASPAC
      url: https://github.com/SASPAC
  #   - title: PharmaForest
  #     url: https://github.com/PharmaForest
```


## GitHub Actions Deployment
This repository includes a GitHub Actions workflow (`.github/workflows/deploy.yml`) that automatically builds and deploys your documentation to GitHub Pages.
You can use it in your repository by copying this file or generating it with the macro functionality.

### Setup Steps:

1. **Enable GitHub Pages**
   - Go to your repository Settings → Pages
   - Under "Build and deployment", set Source to **GitHub Actions**

2. **Push Your Documentation**

3. **Automatic Deployment**
   - The workflow triggers on push to the `main` branch
   - It automatically detects whether you're using Jupyter Book (myst.yml) or Sphinx (conf.py)
   - Builds the HTML documentation
   - Deploys to GitHub Pages

4. **Access Your Documentation**
   - After deployment completes, your documentation will be available at:
   - `https://<username>.github.io/<repository-name>/`

### Using the Macro
The `%packagedoc` macro includes functionality to generate this workflow file. Set `ghActions=1` to enable generation.  
If docsLocation is a subdirectory of filesLocation, the repository root is determined to be filesLocation; otherwise, it is determined to be the parent directory of filesLocation.  
The `.github/workflows` directory is created in the repository root, and the workflow configuration file is generated there.

To output to an arbitrary location instead of `.github/workflows`, specify `wfLocation=`.
When using SAS OnDemand, the `.github` directory cannot be accessed from the file system, so you need to output to a different folder.


In [ ]:

%packagedoc(
  filesLocation=&pkgDir.,
  docsLocation=&pkgDocsOut.,
  docDepth=2,
  engine=MYST,
  newConf=1,
  ghActions=1,
  wfLocation=&pkgDocsOut.
);

In [ ]:
/* View the generated workflow */
title1 "Generated deploy.yml";
data _null_;
  infile "&pkgDocsOut./deploy.yml" ;
  file print ;
  input ;
  put _infile_;
  if _n_ = 5 then stop;
run;
title;

## Customizing Output
For detailed customization such as the order and group labels in `%generateMD`, you can modify the output dataset from `%collectfiles`.  
The `type` variable is set by default to folder number or hierarchy_foldername, but if you set it beforehand, that value will be used.  

Since `%packagedoc` also accepts a dataset of file information, you can customize it by using the intermediate macro `%_collectFilesWithCodes` and adjusting variable values.  

For HTML conversion customization, since this package only generates source files, please refer to [Sphinx](https://www.sphinx-doc.org/en/master/index.html) or [MyST](https://mystmd.org/) customization methods.

## Selective File Exclusion
Files excluded from the file list dataset will not be included in the documentation.  
Additionally, using the exclusion comment `/*##ExcludeFromMarkdownDoc##*/` will exclude content from package markdown but allow output in Sphinx or MyST.
Similarly, exclusions using `/*##ExcludeFromMySTDoc##*/` and `/*##ExcludeFromSphinxDoc##*/` are also available.  

## Compatibility with SPF Logic
For stability, SPF logic is used where possible, but there are the following differences:

The process first collects folder information using logic common to general folders, then filters it to package-specific information. Cases that don't cause issues during documentation generation may fail validation during package creation (e.g., when a `README.md` is included in the package folder).

License file processing is skipped if no license file exists, assuming the package may be under development.  
When generating documentation from a ZIP file, ordering information (folder name numbers) is lost.  
For complete compatibility, customize the file information dataset.